<a href="https://colab.research.google.com/github/qw-4735/PyTorch/blob/main/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98(PyTorch)%EC%9D%98_nn_Embedding().ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

파이토치에서 임베딩 벡터를 사용하는 방법
1. **임베딩 층 (embedding layer)**을 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법 : nn.Embedding() 사용하여 구현

2. **미리 사전에 훈련된 임베딩 벡터(pre-trained word embedding)**들을 가져와 사용하는 방법

### 1.  **임베딩 층 (embedding layer)**을 만들어 훈련 데이터로부터 처음부터 임베딩 벡터를 학습하는 방법

1) 임베딩 층은 룩업 테이블이다.

In [4]:
import torch

In [2]:
train_data = 'you need to know how to code'

# 중복을 제거한 단어들의 집합인 단어 집합 생성
word_set = set(train_data.split())

# 단어 집합의 각 단어에 고유한 정수 맵핑
vocab = {word: i+2 for i, word in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1
print(vocab)

{'code': 2, 'need': 3, 'you': 4, 'know': 5, 'to': 6, 'how': 7, '<unk>': 0, '<pad>': 1}


In [5]:
# 단어 집합의 크기만큼의 행을 가지는 테이블 생성
embedding_table = torch.FloatTensor([
                               [ 0.0,  0.0,  0.0],
                               [ 0.0,  0.0,  0.0],
                               [ 0.2,  0.9,  0.3],
                               [ 0.1,  0.5,  0.7],
                               [ 0.2,  0.1,  0.8],
                               [ 0.4,  0.1,  0.1],
                               [ 0.1,  0.8,  0.9],
                               [ 0.6,  0.1,  0.1]])

2) 임베딩 층 사용하기 : nn.Embedding()을 사용할 경우

In [6]:
train_data = 'you need to know how to code'

# 중복을 제거한 단어들의 집합인 단어 집합 생성
word_set = set(train_data.split())

# 단어 집합의 각 단어에 고유한 정수 맵핑
vocab = {tkn: i+2 for i, tkn in enumerate(word_set)}
vocab['<unk>'] = 0
vocab['<pad>'] = 1

In [7]:
# nn.Embedding()을 사용하여 학습가능한 임베딩 테이블을 만든다.
import torch.nn as nn
embedding_layer = nn.Embedding(num_embeddings=len(vocab),   # num_embeddings : 임베딩 할 단어들의 개수 ( 단어 집합의 크기)
                               embedding_dim =3,            # embedding_dim : 임베딩 할 벡터의 차원 (사용자가 정해주는 하이퍼 파라미터)
                               padding_idx=1)               # padding_idx : 패딩을 위한 토큰의 인덱스를 알려줌 , 선택적으로 사용하는 인자

In [8]:
print(embedding_layer.weight)   # 단어 집합의 크기의 행을 가지는 임베딩 테이블 생성

Parameter containing:
tensor([[ 1.5196, -0.7165,  0.7309],
        [ 0.0000,  0.0000,  0.0000],
        [-1.2850, -0.3948, -1.1524],
        [-1.9137, -1.4778, -1.3510],
        [-0.9318, -1.2556,  1.7311],
        [ 0.3184, -0.7792,  1.1011],
        [ 0.2000, -0.4754,  1.6249],
        [ 0.6655,  0.3876,  0.1850]], requires_grad=True)


### 2. 사전 훈련된 워드 임베딩(Pretrained Word Embedding)
  훈련 데이터가 부족한 경우 모델에 파이토치의 nn.Embedding()으로 해당 문제에 충분히 특화된 임베딩 벡터를 만들어내는 것이 쉽지 않을 수 있다.


이 경우, 해당 문제에 특화된 것은 아니지만 보다 일반적이고 보다 많은 훈련 데이터로 이미 Word2Vec이나 GloVe 등으로 학습되어져 있는 임베딩 벡터들을 사용하는 것이 성능의 개선을 가져올 수 있다.

1) IMDB 리뷰 데이터를 훈련 데이터로 사용하기
- 실습을 위해서는 사전 훈련된 임베딩 벡터들을 맵핑시킬 대상인 훈련 데이터가 필요

In [9]:
from torchtext import data, datasets

In [2]:
!pip install -U torch==1.8.0 torchtext==0.9.0

# Reload environment
exit()

     |████████████████████████████████| 735.5 MB 13 kB/s 
     |████████████████████████████████| 7.1 MB 43.3 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.12.0
    Uninstalling torchtext-0.12.0:
      Successfully uninstalled torchtext-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.0 which is incompatible.


In [1]:
import torchtext
import torch
from torchtext.legacy import data
from torchtext.legacy import datasets

In [4]:
# 필드 객체 정의
TEXT = data.Field(sequential=True, batch_first=True, lower=True)
LABEL = data.Field(sequential=False, batch_first=True)

In [5]:
trainset, testset = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:04<00:00, 17.3MB/s]


In [6]:
print(len(trainset))  # 훈련 데이터의 크기

25000


In [7]:
print(vars(trainset[0]))   # 첫번째 샘플 출력

{'text': ['if', 'you', 'are', 'uninitiated', 'to', 'the', 'gundam', 'world,', 'this', 'is', 'a', 'good', 'place', 'to', 'start.', 'if', 'you', 'are', 'burned', 'out', 'on', 'star', 'wars', 'or', 'star', 'trek,', 'here', 'is', 'a', 'compelling,', 'realistic', 'sci-fi', 'series', 'you', 'can', 'become', 'immersed', 'in.', 'not', 'the', 'simplistic', 'boy-saves-world-in-giant', 'robot', 'story', 'you', 'might', 'have', 'expected,', 'but', 'rather', 'a', 'complex,', 'emotionally', 'compelling', 'space', 'war', 'drama', 'where', 'the', 'line', 'between', 'the', '"good"', 'and', '"bad"', 'guys', 'is', 'decidedly', 'less', 'than', 'distinct.<br', '/><br', '/>gundam', '0080', 'focuses', 'on', 'the', 'story', 'of', 'al', 'izuruha,', 'a', 'young,', 'naive', 'boy', 'living', 'in', 'a', 'neutral', 'space', 'colony.', 'he', 'spends', 'his', 'days', 'daydreaming', 'about', 'mobile', 'suits', 'and', 'playing', 'war', 'with', 'his', 'friends.', 'during', 'the', 'course', 'of', 'this', 'series,', 'al',

2) 토치텍스트를 사용한 사전 훈련된 워드 임베딩

2-1) 사전 훈련된 Word2Vec 모델 확인하기
- 여기서는 앞서 '영어/한국어 Word2Vec 훈련하기 챕터'에서 만들어두었던 'eng_w2v' 모델을 사용

In [9]:
from gensim.models import KeyedVectors

In [10]:
word2vec_model = KeyedVectors.load_word2vec_format('eng_w2v')

FileNotFoundError: ignored